In [35]:
import os,sys
import json
import random
random.seed = 1024
import pandas as pd
from tqdm import tqdm

In [36]:
{
    "instruction": "人类指令（必填）",
    "input": "人类输入（选填）",
    "output": "模型回答（必填）",
    "system": "系统提示词（选填）",
    "history": [
      ["第一轮指令（选填）", "第一轮回答（选填）"],
      ["第二轮指令（选填）", "第二轮回答（选填）"]
    ]
  }

{'instruction': '人类指令（必填）',
 'input': '人类输入（选填）',
 'output': '模型回答（必填）',
 'system': '系统提示词（选填）',
 'history': [['第一轮指令（选填）', '第一轮回答（选填）'], ['第二轮指令（选填）', '第二轮回答（选填）']]}

In [37]:
def find_files(folder_path, end_strs):
    txt_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if len([1 for end_str in end_strs if file.endswith(end_str)]) > 0:
                txt_files.append(os.path.join(root, file))
    return txt_files

In [38]:
data_top_dir = '/root/a100_nas_lvbo/peixunban/002754_lvbo/SFT/00_dataset/tulu-3-personas-math'
data_files = find_files(data_top_dir, ['parquet'])
data_files

['/root/a100_nas_lvbo/peixunban/002754_lvbo/SFT/00_dataset/tulu-3-personas-math/data/train-00001-of-00002.parquet',
 '/root/a100_nas_lvbo/peixunban/002754_lvbo/SFT/00_dataset/tulu-3-personas-math/data/train-00000-of-00002.parquet']

In [39]:
sft_data = []
for data_file in data_files:
    df = pd.read_parquet(data_file)
    for i, row in tqdm(df.iterrows()):
        messages = list(row['messages'])
        id = row['id']
        msg_history = []
        begin = 0 if messages[0]['role'] == 'user' else 1
        for i in range((len(messages)) // 2):
            user_i = begin + i*2
            syst_i = user_i + 1
            if max(syst_i, user_i) >= len(messages):
                continue
            input = messages[user_i]['content']
            output = messages[syst_i]['content']
            if messages[user_i]['role'] != 'user' or messages[syst_i]['role'] != 'assistant':
                continue
            msg_history.append([input, output])
        if len(msg_history) == 0:
            continue
        sample = {
            'instruction': msg_history[-1][0],
            'input': '',
            'output': msg_history[-1][1],
            "history": msg_history[:-1],
            'source': id
        }
        sft_data.append(sample)

74980it [00:14, 5329.77it/s]
74980it [00:14, 5326.22it/s]


In [40]:
target_output_path="/root/a100_nas_lvbo/peixunban/002754_lvbo/SFT/00_dataset/tulu-3-personas-math-149K.jsonl"

In [41]:
len(sft_data)
sft_data[100:100+10]
filtered_data = [item for item in sft_data if len(item.get("history", [])) > 0]
filtered_data

[]

In [43]:
sft_data_json = []
for sample in tqdm(sft_data):
    sft_data_json.append(json.dumps(sample, ensure_ascii=False))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 149960/149960 [00:22<00:00, 6536.72it/s]


In [44]:
with open(target_output_path, 'w') as fout:
    for line in sft_data_json:
        fout.write(f'{line}\n')
    fout.flush()

In [45]:
data_top_dir = '/root/a100_nas_lvbo/peixunban/002754_lvbo/SFT/00_dataset/tulu-3-personas-math-grade'
data_files = find_files(data_top_dir, ['parquet'])
data_files

['/root/a100_nas_lvbo/peixunban/002754_lvbo/SFT/00_dataset/tulu-3-personas-math-grade/data/train-00000-of-00001.parquet']

In [47]:
target_output_path1="/root/a100_nas_lvbo/peixunban/002754_lvbo/SFT/00_dataset/tulu-3-personas-math-grade-50k.jsonl"

In [48]:
sft_data_1 = []
for data_file in data_files:
    df = pd.read_parquet(data_file)
    for i, row in tqdm(df.iterrows()):
        messages = list(row['messages'])
        id = row['id']
        msg_history = []
        begin = 0 if messages[0]['role'] == 'user' else 1
        for i in range((len(messages)) // 2):
            user_i = begin + i*2
            syst_i = user_i + 1
            if max(syst_i, user_i) >= len(messages):
                continue
            input = messages[user_i]['content']
            output = messages[syst_i]['content']
            if messages[user_i]['role'] != 'user' or messages[syst_i]['role'] != 'assistant':
                continue
            msg_history.append([input, output])
        if len(msg_history) == 0:
            continue
        sample = {
            'instruction': msg_history[-1][0],
            'input': '',
            'output': msg_history[-1][1],
            "history": msg_history[:-1],
            'source': id
        }
        sft_data_1.append(sample)

49980it [00:09, 5117.19it/s]


In [51]:
sft_data_json_1 = []
for sample in tqdm(sft_data_1):
    sft_data_json_1.append(json.dumps(sample, ensure_ascii=False))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49980/49980 [00:03<00:00, 14515.99it/s]


In [52]:
with open(target_output_path1, 'w') as fout:
    for line in sft_data_json_1:
        fout.write(f'{line}\n')
    fout.flush()

In [34]:
len(sft_data)
sft_data[100:100+10]

[{'instruction': "A young entrepreneur named Alex believes that the phases of the moon have an impact on their business decisions. Alex decides to consult an astrologer to plan the launch of a new product. The astrologer suggests launching the product in 7 days, during the waxing crescent phase, which is believed to bring growth and success. Alex also wants to align the launch with a day when the moon is at its highest position in the sky, which happens every 3 days.\n\nIf today is Monday, calculate the day of the week Alex should launch the product to align with both the waxing crescent phase and the moon's highest position, ensuring the launch happens in 7 days.",
  'input': '',
  'output': "To solve Alex's problem, we need to determine the day of the week 7 days from today, which is Monday, and check if that day also coincides with a day when the moon is at its highest position in the sky, which occurs every 3 days.\n\nLet's break this down step-by-step:\n\n1. **Determine the day of

In [1]:
filtered_data = [item for item in sft_data if len(item.get("history", [])) > 0]
filtered_data

NameError: name 'sft_data' is not defined